In [2]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

### accessibility score: 
- distance to airport 
- distance to train stations
- altitude?

In [3]:
df_municipalities = pd.read_csv("../data/processed/filtered_municipalities.csv")
df_airports = pd.read_csv("../data/processed/filtered_airports.csv")
df_railway = pd.read_csv("../data/processed/filtered_trains.csv")

In [4]:
df_municipalities.head()

,province,municipality_name,population,male,female,municipality_name_clean,province_clean,altitude,longitude,latitude,cmun
0,Álava,Alegría-Dulantzi,2971,1531,1440,alegria-dulantzi,alava,561.6857,-2.513507,42.84149,1001
1,Álava,Amurrio,10330,5149,5181,amurrio,alava,219.6910,-3.001022,43.05265,1002
2,Álava,Aramaio,1381,709,672,aramaio,alava,381.8797,-2.566000,43.05400,1003
3,Álava,Artziniega,1856,913,943,artziniega,alava,196.9808,-3.128209,43.12220,1004
4,Álava,Armiñón,247,127,120,arminon,alava,463.5815,-2.872574,42.72305,1006


In [5]:
df_airports.head()

,airport_id,airport_name,latitude,longitude
0,1,Aeropuerto de Jerez,36.743828,-6.062658
1,2,Aeropuerto de Vigo,42.225740,-8.630491
2,3,Aeropuerto de Badajoz,38.889845,-6.821121
3,4,Aeropuerto de Granada - Jaén F.G.L.,37.186826,-3.778613
4,5,Aeropuerto de Almería,36.844781,-2.371234


In [6]:
df_railway.sample(15)

,station_code,station_name,latitude,longitude,province_clean,municipality_clean
302,60202,Castillejo-Añover,39.949588,-3.738456,madrid,aranjuez
1,11212,Agurain/Salvatierra De Alava,42.846437,-2.389227,alava,agurain
474,76002,Vilabella,41.239554,1.317492,tarragona,vilabella
309,55012,Cortes De La Frontera,36.597359,-5.332795,malaga,cortes de la frontera
105,37300,Puertollano,38.691411,-4.111611,ciudad real,puertollano
566,70501,Cetina,41.295900,-1.959889,zaragoza,cetina
359,11006,Quintana Del Puente,42.080031,-4.198594,palencia,quintana del puente
82,35402,Arroyo De Malpartida,39.468073,-6.534453,caceres,caceres
52,35405,San Vicente De Alcantara,39.381540,-7.117066,badajoz,san vicente de alcantara
346,22005,Os Peares,42.451166,-7.735629,ourense,"peroxa, a"


In [11]:
def find_nearest_airport(row):
    municipality_location = (row['latitude'], row['longitude'])
    # Calculate distance from the town to every airport
    distances = df_airports.apply(lambda x: geodesic(municipality_location, (x['latitude'], x['longitude'])).kilometers, axis=1)
    # Find the index of the airport with the smallest distance
    nearest_index = distances.idxmin()
    # Retrieve the minimum distance and corresponding airport ID
    nearest_distance = distances[nearest_index]
    nearest_airport = df_airports.loc[nearest_index, 'airport_name']
    return pd.Series([nearest_distance, nearest_airport])



In [12]:
# Apply the function to each row in the towns DataFrame and create two new columns
df_municipalities[['closest_distance', 'nearest_airport']] = df_municipalities.apply(find_nearest_airport, axis=1)


In [13]:
df_municipalities.head()

,province,municipality_name,population,male,female,municipality_name_clean,province_clean,altitude,longitude,latitude,cmun,closest_distance,nearest_airport
0,Álava,Alegría-Dulantzi,2971,1531,1440,alegria-dulantzi,alava,561.6857,-2.513507,42.84149,1001,18.273217,Aeropuerto de Vitoria
1,Álava,Amurrio,10330,5149,5181,amurrio,alava,219.6910,-3.001022,43.05265,1002,28.762394,Aeropuerto de Bilbao
2,Álava,Aramaio,1381,709,672,aramaio,alava,381.8797,-2.566000,43.05400,1003,23.639652,Aeropuerto de Vitoria
3,Álava,Artziniega,1856,913,943,artziniega,alava,196.9808,-3.128209,43.12220,1004,26.840606,Aeropuerto de Bilbao
4,Álava,Armiñón,247,127,120,arminon,alava,463.5815,-2.872574,42.72305,1006,20.846277,Aeropuerto de Vitoria


In [ ]:
def find_nearest_location(row, df_location):
    municipality_location = (row['latitude'], row['longitude'])
    # Calculate distance from the town to every airport
    distances = df_location.apply(lambda x: geodesic(municipality_location, (x['latitude'], x['longitude'])).kilometers, axis=1)
    # Find the index of the airport with the smallest distance
    nearest_index = distances.idxmin()
    # Retrieve the minimum distance and corresponding airport ID
    nearest_distance = distances[nearest_index]
    nearest_location = df_location.loc[nearest_index, 'airport_name']
    return pd.Series([nearest_distance, nearest_location])

# Apply the function to each row in the towns DataFrame and create two new columns
df_municipalities[['closest_distance', 'nearest_airport']] = df_municipalities.apply(find_nearest_location(df_location=df_airports), axis=1)
